Présentation de la base de données


Nous avons exporté les données de l'OCDE dans une base de données que nous avons reformatée, afin d'avoir une indexation multiple au niveau des colonnes : un premier niveau pour les pays et ensuite pour chaque pays les différents indicateurs choisis. Nous avons aussi ajouté des indicateurs concernant les inégalités, à savoir le top 50% du revenu et le top 10% du revenu. Néanmoins nous nous posons quelques questions concernant ces indicateurrs. 1) Le coefficient de Gini est disponible mais le calcul n'est fait que par année (car il nous semble qu'il varie peu). Faut-il alors qu'on le calcule à la main pour chaque trimestre, l'utiliser tel quel (par annnée), ou ne pas l'utiliser du tout? 2) Est-il important de savoir si la part des revenus est bruts ou non, dans le mesure où l'on cherche à évaluer l'impact des politiques fiscales de Trump?

In [55]:
import csv

In [93]:
import pandas as pd

ocde_df = pd.read_csv(r"https://raw.githubusercontent.com/Adrien-Mcode/Stat-app-Trumpnomics/main/Tableaux_csv/ocde2.csv?token=ASA6PWRCJSN5UPTN4K77SYLACU44O")

ocde_df.rename(columns={'Unnamed: 0':'Variables'}, inplace=True)

ind_tuple = list(zip(ocde_df['Pays'], ocde_df['Variables']))
new_index = pd.MultiIndex.from_tuples(ind_tuple, names=["Pays", "Variables"])

tocde = ocde_df.T.copy()
tocde.columns = new_index
tocde.drop(['Variables', 'Pays'], inplace=True)

In [112]:
import csv
bottom_50 = pd.read_csv(r"https://raw.githubusercontent.com/Adrien-Mcode/Stat-app-Trumpnomics/main/Donnees%20inegalite/top_bottom_income/WID_Data_Metadata/bottom_50_income.csv?token=ASA6PWUTFBTHQOQKNCTKW3DACVADK",
                       header = 1,
                       sep=';')

In [115]:
import csv
top_10 =pd.read_csv(r"https://raw.githubusercontent.com/Adrien-Mcode/Stat-app-Trumpnomics/main/Donnees%20inegalite/top_bottom_income/WID_Data_Metadata/top_10_income.csv?token=ASA6PWRMIM3ZEUEIJW6LKGDACVAMC",
                   header = 1,
                   sep=';')

In [99]:
df_ocde = pd.read_csv(r"https://raw.githubusercontent.com/Adrien-Mcode/Stat-app-Trumpnomics/main/Tableaux_csv/ocde2.csv?token=ASA6PWWQBJ522O3JJD4YY73ACU6KA")

In [113]:
bottom_50.columns

Index(['Percentile', 'Year',
       'sptinc_z_AT\nPre-tax national income\nBottom 50% | share | adults | equal split\nAustria',
       'sptinc_z_GB\nPre-tax national income\nBottom 50% | share | adults | equal split\nUnited Kingdom',
       'sptinc_z_US\nPre-tax national income\nBottom 50% | share | adults | equal split\nUSA',
       'sptinc_z_TR\nPre-tax national income\nBottom 50% | share | adults | equal split\nTurkey',
       'sptinc_z_BE\nPre-tax national income\nBottom 50% | share | adults | equal split\nBelgium',
       'sptinc_z_CA\nPre-tax national income\nBottom 50% | share | adults | equal split\nCanada',
       'sptinc_z_DK\nPre-tax national income\nBottom 50% | share | adults | equal split\nDenmark',
       'sptinc_z_FI\nPre-tax national income\nBottom 50% | share | adults | equal split\nFinland',
       'sptinc_z_FR\nPre-tax national income\nBottom 50% | share | adults | equal split\nFrance',
       'sptinc_z_DE\nPre-tax national income\nBottom 50% | share | adults | equa

In [116]:
import pandas as pd
import csv


pays_ocde = {"Germany" :'DEU',"Australia" :'AUS',"Austria":'AUT',"Belgium":'BEL',
             "Canada":'CAN',"Denmark":'DNK',"Spain":'ESP',"Finland":'FIN',
             "France":'FRA',"Greece":'GRC',"Ireland":'IRL',"Italy":'ITA',
             "Japan":'JPN',"Luxembourg":'LUX',"Norway":'NOR',"New-Zealand":'NZL',
             "Netherlands":'NLD',"Portugal":'PRT',"United Kingdom":'GBR',
             "Sweden":'SWE',"Switzerland":'CHE',"Turkey":'TUR',"United-States":'USA'}



df_ocde.rename(columns={'Unnamed: 0':'Variables'}, inplace=True)

ind_tuple = list(zip(df_ocde['Pays'], df_ocde['Variables']))
new_index = pd.MultiIndex.from_tuples(ind_tuple, names=["Pays", "Variables"])
#mise en place du système de double indice
tocde = df_ocde.T.copy()
tocde.columns = new_index
tocde.drop(['Variables', 'Pays'], inplace=True)
#tocde = tocde.drop(['2020-Q1','2020-Q2','2020-Q3','2020-Q4','2021-Q1'])
#renommage des colonnes de bottom_50 et top_10
bottom_50.columns = ['Variables', 'Year'] + sorted(pays_ocde.keys(), key=str)
top_10.columns = ['Variables', 'Year'] + sorted(pays_ocde.keys(), key=str)

#on prépare bottom_50 pour la fusion
bottom_50 = bottom_50.drop('Variables', axis=1)
for i in range (1,3) : 
    bottom_50 = pd.concat([bottom_50,bottom_50]) 
    
annee = []
for j in range (1,5):
    for i in range (0,29):
        annee.append(str(1991+i)+'-Q'+str(j))
bottom_50['Year'] = annee

#Mise en place double index pour bottom_50 :
bottom_50 = bottom_50.set_index('Year')

bottom_50['Variables'] = 'income p0p50'

ind_tuple = list(zip(bottom_50.columns, bottom_50['Variables']))

new_index = pd.MultiIndex.from_tuples(ind_tuple, names=["Pays", "Variables"])

bottom_50.columns = new_index

bottom_50 = bottom_50.drop(('Variables','income p0p50'), axis = 1)

tocde = tocde.merge(right=bottom_50, how='outer',
                    left_index=True,
                    right_index=True).sort_index(axis = 0).sort_index(axis = 1)

#on prépare top_10 pour la fusion :
top_10 = top_10.drop('Variables', axis = 1)
for i in range (1,3) : 
    top_10 = pd.concat([top_10,top_10])    
    
annee = []
for j in range (1,5):
    for i in range (0,29):
        annee.append(str(1991+i)+'-Q'+str(j))
top_10['Year'] = annee

#Mise en place double index pour top_10 :
top_10 = top_10.set_index('Year')

top_10['Variables'] = 'income p90p100'

ind_tuple = list(zip(top_10.columns, top_10['Variables']))

new_index = pd.MultiIndex.from_tuples(ind_tuple, names=["Pays", "Variables"])

top_10.columns = new_index

top_10 = top_10.drop(('Variables','income p90p100'), axis=1)

tocde = tocde.merge(right = top_10,
                    how='outer',
                    left_index=True,
                    right_index=True).sort_index(axis = 0).sort_index(axis = 1)

qs = tocde.index.str.replace(r'(Q\d) (\d+)', r'\2-\1')

tocde['date'] = pd.PeriodIndex(qs, freq='Q').to_timestamp()


tocde = tocde.reindex(index=tocde['date'])
tocde = tocde.drop("date", axis=1)

Nous allons désormais effectuer des tests de stationnarité sur les différentes variables. Nous appliquons deux tests  : KPSS qui a pour hypothèse nulle H0:"La série est stationnaire" et le test AFD dont l'hypothèse nulle est H0 : "La série est non stationnaire". NB : nous n'avons pas désaisonnalisé les séries avant d'effectuer les tests car nosu ne savions pas si cela éait nécessaire.

 Dans un premier temps nous nous intéressons au PIB des Etats-Unis. D'abord le test KPSS :

In [122]:
serie1=tocde['USA']['PIB']

date
1990-01-01    9358.29
1990-04-01    9392.25
1990-07-01     9398.5
1990-10-01    9312.94
1991-01-01    9269.37
               ...   
2018-10-01    18813.9
2019-01-01    18950.3
2019-04-01    19020.6
2019-07-01    19141.7
2019-10-01      19254
Name: PIB, Length: 120, dtype: object

In [123]:
from statsmodels.tsa.stattools import kpss
def kpss_test(series, **kw):    
    statistic, p_value, n_lags, critical_values = kpss(series, **kw)
    # Format Output
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
    print(f'Result: The series is {"not " if p_value < 0.05 else ""}stationary')

kpss_test(serie1)

KPSS Statistic: 0.9556147530640225
p-value: 0.01
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is not stationary


C:\Users\duran\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1661: FutureWarning: The behavior of using lags=None will change in the next release. Currently lags=None is the same as lags='legacy', and so a sample-size lag length is used. After the next release, the default will change to be the same as lags='auto' which uses an automatic lag length selection method. To silence this warning, either use 'auto' or 'legacy'
  warn(msg, FutureWarning)
C:\Users\duran\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1685: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)


La série n'est pas stationnaire au seuil de 5%, nous allons donc appliquer le test à la série intégrée à l'ordre 1.

In [124]:
serie2 = serie1 - serie1.shift().fillna(0)
serie2.drop(pd.Timestamp('1990-01-01 00:00:00'), inplace=True)
kpss_test(serie2)

KPSS Statistic: 0.08667738001103942
p-value: 0.1
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary


C:\Users\duran\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1687: InterpolationWarning: p-value is greater than the indicated p-value
  warn("p-value is greater than the indicated p-value", InterpolationWarning)


Maintenant le test AFD

In [125]:
from statsmodels.tsa.stattools import adfuller
def adf_test(series, **kw):    
    statistic, p_value, n_lags, nobs, critical_values, icbest = adfuller(series, **kw)
    # Format Output
    print(f'ADF Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
    print(f'Result: The series is {"not " if p_value > 0.05 else ""}stationary')

adf_test(serie1)
adf_test(serie2)

ADF Statistic: -0.03066086875171472
p-value: 0.9559218024233863
num lags: 2
Critial Values:
   1% : -3.487517288664615
   5% : -2.8865777180380032
   10% : -2.5801239192052012
Result: The series is not stationary
ADF Statistic: -5.015879029087158
p-value: 2.0642028929741757e-05
num lags: 1
Critial Values:
   1% : -3.487517288664615
   5% : -2.8865777180380032
   10% : -2.5801239192052012
Result: The series is stationary


Comme pour le test KPSS, la série intégrée à l'ordre 1 est stationnaire.

In [127]:
import seaborn as sns

#sns.set_theme(style="darkgrid")

PIB_graph = sns.lineplot(data=serie2, x='date', y=serie2)
PIB_graph.set_title("Évolution stationnarisée du PIB américain depuis 1990")

ValueError: Could not interpret input 'date'